<a href="https://colab.research.google.com/github/Swapnil-Parihar/Text-Summarization-PEGASUS/blob/main/Text_Summarization_using_PEGASUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets rouge_score -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [2]:
from transformers import pipeline, PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [3]:
device= "cuda" if torch.cuda.is_available() else "cpu"
device #it shows that i have selected GPU in runtime hence cuda, else cpu

'cuda'

PEGASUS

In [4]:
Pegasus_dailymail = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(Pegasus_dailymail)
Pegasus = PegasusForConditionalGeneration.from_pretrained(Pegasus_dailymail).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Dataset : CNN

In [ ]:
dataset_cnn = load_dataset('cnn_dailymail', '3.0.0')

In [ ]:
Sample_article = CNN_article_list[7]
Sample_article

"New York Governor Andrew Cuomo has become the first governor in five years to visit Cuba, following President Obama's ease on trade and travel to  the communist nation. The Governor is heading a delegation of 18 academics and business leaders visiting the island in the wake of the December announcement that the US and Cuba would restore diplomatic relations after more than a half century of hostility and confrontation. Today, Cuomo met with Cuba's top officials for U.S. relations along with executives from Jetblue, MasterCard, Pfizer and other New York-based companies. Scroll down for video . Thumbs up: Andrew Cuomo has become the first American governor to visit the island since the recent thaw in relations with the communist nation, is meant to foster greater ties between New York and Cuba . Local tradition: Governor Cuomo enjoys a mojito during a meeting at the Hotel Nacional in Havana on the first evening of the delegation's visit . Business first: Cuomo has met with Cuba's top of

In [ ]:
# Tokenize the sample article
input = tokenizer.encode("summarize: " + Sample_article, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate the summary tokens
summary = Pegasus.generate(input, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True).to(device)

# Decode the summary
summary = tokenizer.decode(summary[0], skip_special_tokens=True)
ref_summary = summary.replace("<n>"," ")
ref_summary

"New York Governor Andrew Cuomo has become the first American governor to visit Cuba since the recent thaw in relations with the communist nation. Cuomo is heading a delegation of 18 academics and business leaders visiting the island in the wake of the December announcement that the US and Cuba would restore diplomatic relations. Today, Cuomo met with Cuba's top officials for U.S. relations along with executives from Jetblue, MasterCard, Pfizer and other New York-based companies."

In [ ]:
summary.size()

torch.Size([1, 98])

Evaluation of summarization on random samples

In [ ]:
import random
cnn_articles= [random.randint(1, 10000) for _ in range(10)]
print(cnn_articles)

[2722, 7305, 6163, 4438, 84, 7429, 3431, 5087, 7496, 8810]


In [ ]:
CNN_article_list=[]
for i in cnn_articles:
  cnn_sample = dataset_cnn["test"][i]["article"][:]
  CNN_article_list.append(cnn_sample)

In [ ]:
CNN_highlight_list=[]
for i in cnn_articles:
  cnn_summry=dataset_cnn["test"][i]["highlights"][:]
  highlight = cnn_summry.replace("\n"," ")
  CNN_highlight_list.append(highlight)

In [ ]:
Pegasus_generated_summary=[]
for i in CNN_article_list:
  # Tokenize the input article
  inputs = tokenizer.encode("summarize: " + i, return_tensors="pt", max_length=1024, truncation=True).to(device)

  # Generate the summary
  summary_ids = Pegasus.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True).to(device)

  # Decode the summary
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  ref_summary = summary.replace("<n>"," ")
  Pegasus_generated_summary.append(ref_summary)

In [ ]:
di={}
for i in range(len(Pegasus_generated_summary)):
  di[CNN_highlight_list[i]]=Pegasus_generated_summary[i]

In [ ]:
rouge_score = load_metric('rouge')

<ipython-input-27-d7f30f715e28>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_score = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ROGUE 1 score for CNN

In [ ]:
record1 = []
key_list=list(di.keys())
for i in range(len(di)):
  keys=key_list[i]
  values=di[keys]
  rouge_score .add(prediction = values, reference = keys )
  score = rouge_score .compute()
  rouge_dict1 = score["rouge1"].mid.fmeasure
  record1.append(rouge_dict1)
record1

[0.37499999999999994,
 0.3826086956521739,
 0.40336134453781514,
 0.4736842105263158,
 0.49411764705882355,
 0.3392857142857143,
 0.44,
 0.5666666666666668,
 0.2830188679245283,
 0.4888888888888889]

ROGUE 2 score for CNN

In [ ]:
record2 = []
key_list=list(di.keys())
for i in range(len(di)):
  keys=key_list[i]
  values=di[keys]
  rouge_score .add(prediction = values, reference = keys )
  score = rouge_score .compute()
  rouge_dict2 = score["rouge2"].mid.fmeasure
  record2.append(rouge_dict2)
record2

[0.10638297872340424,
 0.08849557522123894,
 0.11965811965811966,
 0.23214285714285712,
 0.3373493975903615,
 0.1818181818181818,
 0.24489795918367346,
 0.4406779661016949,
 0.057692307692307696,
 0.29545454545454547]

ROGUE L score for CNN

In [ ]:
recordL = []
key_list=list(di.keys())
for i in range(len(di)):
  keys=key_list[i]
  values=di[keys]
  rouge_score .add(prediction = values, reference = keys)
  score = rouge_score .compute()
  rouge_dict2 = score["rougeL"].mid.fmeasure
  recordL.append(rouge_dict2)
recordL

[0.22916666666666669,
 0.15652173913043477,
 0.20168067226890757,
 0.3157894736842105,
 0.28235294117647053,
 0.26785714285714285,
 0.38000000000000006,
 0.5166666666666667,
 0.1320754716981132,
 0.4222222222222222]